1. В коде из методички реализуйте один или несколько из критериев останова (количество листьев, количество используемых признаков, глубина дерева и т.д.).
2. Для задачи классификации обучить дерево решений с использованием критериев разбиения Джини и Энтропия. Сравнить качество классификации, сделать выводы.
3. [опция]. Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.

-----

Есть желание полностью переписать код, заимствуя некоторые моменты. Отдельно выделю фрагменты, необходимые по заданию.

In [93]:
from sklearn import datasets

import numpy as np
from matplotlib import pyplot as plt
from sklearn import model_selection

In [2]:
from sklearn.tree import DecisionTreeClassifier

In [55]:
class Node:
    '''
    Узел решающего дерева, храняющий определитель признака и порогового значения, а также
    направления на следующие узлы.
    '''
    def __init__(self, index, thershold, true_branch, false_branch):
        self.index = index  # индекс признака
        self.thershold = thershold
        self.true_branch = true_branch
        self.false_branch = false_branch


class Leaf:
    '''
    Узлы, в которых происходит выдача предсказания и дальше дерево не строится
    '''
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        classes = dict()
        for label in self.labels:
            classes[label] = classes.get(label, 0) + 1
        prediction = max(classes, key=classes.get)
        return prediction  

In [81]:
class DecisionTree:
    '''
    Классификатор, построенный на алгоритме решающего дерева.
    
    
    Parameters:
    -----------
    predict_type: str
        
    max_depth: int
        
    min_leaf: int
        
    criterion: {"gini", "entropy"}
        Критерий разбиения значения в разные узлы для классификации.
        Для задачи регрессии используется оценка дисперсии признаков и атрибут не учитывается.
    max_leafes_number: int
        
    
    '''
    def __init__(self,
                 predict_type: str='classification',
                 max_depth: int=None,
                 min_leaf: int=1,
                 criterion: str='gini',
                 max_leafes_number: int=None):
        # перечисленные параметры являются параметрами останова, настраиваемыми далее. 1 задание
        self.__predict_type = predict_type
        self.__max_depth = max_depth
        self.__min_leaf = min_leaf
        if predict_type == 'regression':
            criterion = 'variance'
        self.__criterion = criterion
        self.__max_leafes_number = max_leafes_number
        self.__n_leafs = 0
    
    def _gini(self, labels: np.array):
        '''
        Расчет порогового значения в узле(Node) на основании критерия Джини.
        Первая часть 2 задания
        
        
        Parameters:
        -----------
        labels: array like
            перечень значений по выбранной метрике
        '''        
        labels = np.array(labels)
        classes, size = np.unique(labels, return_counts=True)
        impurity = 1 - ((size / labels.shape)**2).sum()
        return impurity
        
    def _entropy(self, labels: np.array):
        '''
        Расчет порогового значения в узле(Node) на основании энтропии.
        Первая часть 2 задания
        '''
        labels = np.array(labels)
        classes, size = np.unique(labels, return_counts=True)
        p = size / labels.shape
        impurity = - (p * np.log2(p)).sum()
        return impurity

    def _variance(self, labels: np.array):
        '''
        Расчет порогового значения в узле(Node) на основании дисперсии целевой переменной.
        3 задание
        '''
        labels = np.array(labels)
        pass
    
    def _quality(self, left_labels, right_labels, criteria_current):
        p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
        if self.__criterion == 'gini':
            criteria = self._gini
        elif self.__criterion == 'entropy':
            criteria = self._entropy
        return criteria_current - p * criteria(left_labels) - (1 - p) * criteria(right_labels)
    
    def _split(self, data, labels, index, t):
        left = np.where(data[:, index] <= t)
        right = np.where(data[:, index] > t)

        true_data = data[left]
        false_data = data[right]
        true_labels = labels[left]
        false_labels = labels[right]

        return true_data, false_data, true_labels, false_labels
    
    def _find_best_split(self, data, labels):
        if self.__criterion == 'gini':
            criteria = self._gini
        elif self.__criterion == 'entropy':
            criteria = self._entropy
            
        criteria_current = criteria(labels)

        best_quality = 0
        best_t = None
        best_index = None

        n_features = data.shape[1]

        for index in range(n_features):
            t_values = np.unique(data[:, index])

            for t in t_values:
                true_data, false_data, true_labels, false_labels = self._split(data, labels, index, t)
                if min(len(true_data), len(false_data)) < self.__min_leaf:
                    continue

                current_quality = self._quality(true_labels, false_labels, criteria_current)

                if current_quality > best_quality:
                    best_quality, best_t, best_index = current_quality, t, index

        return best_quality, best_t, best_index
    
    def _build_tree(self, data, labels, depth=0):
        quality, t, index = self._find_best_split(data, labels)

        if quality == 0 or (self.__max_depth is not None and depth >= self.__max_depth):
            self.__n_leafs += 1
            return Leaf(data, labels)

        true_data, false_data, true_labels, false_labels = self._split(data, labels, index, t)
        
        true_branch = self._build_tree(true_data, true_labels, depth + 1)
        false_branch = self._build_tree(false_data, false_labels, depth + 1)

        return Node(index, t, true_branch, false_branch)
    
    def _classify_object(self, obj, node):
        if isinstance(node, Leaf):
            answer = node.prediction
            return answer

        if obj[node.index] <= node.thershold:
            return self._classify_object(obj, node.true_branch)
        else:
            return self._classify_object(obj, node.false_branch)
    
    def fit(self, X, y):
        self.__tree = self._build_tree(X, y)
    
    def predict(self, X):
        classes = []
        tree = self.__tree
        for obj in X:
            prediction = self._classify_object(obj, tree)
            classes.append(prediction)
        return classes
    
    def fit_predict(self, X, y):
        self.fit(X, y)
        return self.predict(X)

In [92]:
X_train,X_test, y_train, y_test = datasets.make_classification(n_samples=10, n_features=4)
X_train, y_train


ValueError: not enough values to unpack (expected 4, got 2)

In [89]:
model_gini = DecisionTree(criterion='gini')
model_entropy = DecisionTree(criterion='entropy')

In [90]:
pred_gini = model_gini.fit_predict(X, y)
pred_entropy = model_entropy.fit_predict(X, y)

In [91]:
pred_gini, pred_entropy

([1, 0, 1, 0, 1, 0, 0, 1, 0, 1], [1, 0, 1, 0, 1, 0, 0, 1, 0, 1])

В данном примере особой разницы между использованием разных критериев нет.